In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

thresh = 25
max_diff = 50

a, b, c = None, None, None
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 480)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 320)

if cap.isOpened():
    ret, a = cap.read()
    ret, b = cap.read()
    while ret:
        ret, c = cap.read()
        draw = c.copy() # 출력 영상에 사용할 복제본
        if not ret:
            break
        
        a_gray = cv2.cvtColor(a, cv2.COLOR_BGR2GRAY)
        b_gray = cv2.cvtColor(b, cv2.COLOR_BGR2GRAY)
        c_gray = cv2.cvtColor(c, cv2.COLOR_BGR2GRAY)
        
        # absdiff(a,b) : a - b 하여 최소 0로 만든다 -> 동일하면 검정색, 다르면 흰색
        diff1 = cv2.absdiff(a_gray, b_gray) # 겹치는것은 0 즉 검정색으로 : 안겹친다(=움직인다) -> 255 -> 흰색
        diff2 = cv2.absdiff(b_gray, c_gray)
        
        _, diff1_t = cv2.threshold(diff1, thresh, 255, cv2.THRESH_BINARY) # 스레시홀드로 기준치 초과하면 흰색으로
        _, diff2_t = cv2.threshold(diff2, thresh, 255, cv2.THRESH_BINARY) # 0~24 -> 0 , 24까지는 겹치는 것으로 
        
        # bitwise_and(a,b) : a, b 공통으로 겹치는부분 출력 = a, b 둘다 0이면 0으로(검정색) = a, b 둘중 하나라도 0이아니면 연산 안함(255)
        diff = cv2.bitwise_and(diff1_t, diff2_t) # diff1_t, diff2_t 둘다 0인것만 0으로 : 프레임 3개를 보고 움직임 확인
        
        # 열림 연산으로 noise제거하기 : 생략가능
        k = cv2. getStructuringElement(cv2.MORPH_CROSS, (3,3))
        diff = cv2.morphologyEx(diff, cv2.MORPH_OPEN, k)
        
        diff_cnt = cv2.countNonZero(diff) # 차이가 발생한(255) 픽셀 개수
        if diff_cnt > max_diff:
            nzero = np.nonzero(diff) # diff 2차원 배열에서 차이가 발생한 픽셀의 인덱스(좌표) 얻기
            cv2.rectangle(draw, (min(nzero[1]), min(nzero[0])), (max(nzero[1]), max(nzero[0])), (0,255,0), 2)
            cv2.putText(draw, "Motion Detected", (10, 30), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0,0,255))
        
        stacked = np.hstack((draw, cv2.cvtColor(diff, cv2.COLOR_GRAY2BGR))) # hstack을 위한 차원 맞추기
        cv2.imshow('motion sensor', stacked)
        
        # 한프레임 넘어감
        a = b
        b = c
        # 이후 c는 while문 처음으로 돌아가서 다시 한프레임 읽어온 새로운 c로 배정
        
        if cv2.waitKey(1) & 0XFF ==27:
            break
        cv2.waitKey(1)
    cv2.waitKey(1)
cv2.waitKey(1)


-1